<a href="https://colab.research.google.com/github/Learner890/Python-SCD-Type-1/blob/main/SCD_Type1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sqlalchemy as sal
engine=sal.create_engine('mssql://LAPTOP-P5QL3BL8\SQLEXPRESS/master?driver=ODBC+DRIVER+11+FOR+SQL+SERVER',future=True)
conn=engine.connect()

In [ ]:
def extract():
    df_orders=pd.read_csv(R'C:\Users\nisha\Downloads\orders.txt')
    df_returns=pd.read_csv(R'C:\Users\nisha\Downloads\returns.txt')
    return df_orders,df_returns

def transform(df_orders,df_returns):
    df_tranformed=pd.merge(df_orders,df_returns,how='inner',left_on='order_id',right_on='Order Id')
    return df_tranformed

def load(df_tranformed):
    df_tranformed.to_sql('Order_Final',con=conn,index=False,if_exists='replace')
    conn.commit()

In [ ]:
df_orders,df_returns=extract()
df_tranformed=transform(df_orders,df_returns)
load(df_tranformed)
#print(df_orders,df_returns)

In [ ]:
def prod_extract():
    df_product=pd.read_csv(R'C:\Users\nisha\Downloads\products.txt')
    return df_product
def prod_load(df_product):
    df_product.to_sql('Products',con=conn,index=False,if_exists='replace')
    conn.commit()

In [ ]:
def extract():
    df_product=pd.read_csv(R'C:\Users\nisha\Downloads\products.txt')
    df_product_sql=pd.read_sql_query('SELECT * FROM Products',conn)
    return df_product,df_product_sql

def transform(df_product,df_product_sql):
    df_merged=pd.merge(df_product,df_product_sql,how='left',on='product_id')
    df_insert=df_merged[df_merged['product_name_y'].isna()]
    df_insert_final=df_insert.iloc[:,0:3]
    df_insert_final.columns=df_product_sql.columns
    df_update=df_merged[df_merged['product_name_y'].notna()]
    df_update_final=df_update.iloc[:,0:3]
    df_update_final.columns=df_product_sql.columns
    return df_insert_final,df_update_final
    #df_insert=df_merged[df_merged[]]

def load_stage(df_update_final):
    delete_table=sal.text("truncate table Products_Stg")
    command=conn.execute(delete_table)
    conn.commit()
    df_update_final.to_sql('Products_Stg',con=conn,index=False,if_exists='append')
    conn.commit()

def update():
    query=sal.text("update Products set price=Products_stg.price from Products_stg where Products.product_id=Products_stg.product_id")
    execute=conn.execute(query)
    conn.commit()

def insert(df_insert_final):
    df_insert_final.to_sql('Products',con=conn,index=False,if_exists='append')
    conn.commit()

In [ ]:
df_product,df_product_sql=extract()
df_insert_final,df_update=transform(df_product,df_product_sql)
df_upd_final=load_stage(df_update)
update()
insert(df_insert_final)
load_stage(df_update)
#load(df_output)
#df_output